In [256]:
import polars as pl
import seaborn as sns
from os import path
import requests
from bs4 import BeautifulSoup as bs
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.firefox import GeckoDriverManager
from datetime import datetime
from copy import copy
import time



In [257]:
team_stats = path.join("..", "data", "external", "Team stats.xlsx")

In [258]:
stats_df = pl.read_excel(team_stats, sheet_name="stats_clean", engine="xlsx2csv")

In [259]:
print(stats_df)

shape: (19, 53)
┌──────┬──────────────────────────┬──────┬──────┬───┬─────────────┬──────┬──────────┬──────────┐
│ RNK  ┆ TEAM                     ┆ GP   ┆ W    ┆ … ┆ DF 100 Poss ┆ BLK% ┆ eFG%     ┆ TS%      │
│ ---  ┆ ---                      ┆ ---  ┆ ---  ┆   ┆ ---         ┆ ---  ┆ ---      ┆ ---      │
│ i64  ┆ str                      ┆ i64  ┆ i64  ┆   ┆ f64         ┆ f64  ┆ f64      ┆ f64      │
╞══════╪══════════════════════════╪══════╪══════╪═══╪═════════════╪══════╪══════════╪══════════╡
│ 1    ┆ ALBA Berlin              ┆ 34   ┆ 14   ┆ … ┆ 28.5        ┆ 5.5  ┆ 0.527565 ┆ 0.560578 │
│ 2    ┆ Anadolu Efes Istanbul    ┆ 40   ┆ 23   ┆ … ┆ 27.7        ┆ 7.5  ┆ 0.558292 ┆ 0.596395 │
│ 3    ┆ AS Monaco                ┆ 38   ┆ 21   ┆ … ┆ 28.2        ┆ 5.9  ┆ 0.548507 ┆ 0.582909 │
│ 4    ┆ AX Armani Exchange Milan ┆ 36   ┆ 22   ┆ … ┆ 26.8        ┆ 6.8  ┆ 0.517345 ┆ 0.548064 │
│ …    ┆ …                        ┆ …    ┆ …    ┆ … ┆ …           ┆ …    ┆ …        ┆ …        │
│ 16   ┆ UNICS

In [260]:
def assert_round(driver, season, round):
    elm = driver.find_elements(By.CLASS_NAME, "game-center-select-shared_selectedValue__4xru_")
    corr_season = False
    corr_round = False
    for s in elm:
        if s.get_attribute("aria-labelledby") == "prefix_seasons_select":
            corr_season = s.text == season
        elif s.get_attribute("aria-labelledby") == "prefix_rounds_select":
            corr_round = s.text == round
    return corr_round & corr_season


def load_with_selenium():
    links = []
    # open browser and page
    url="C:\\Programs\\Python36\\BrowersDriver\\geckodriver.exe"
    # Accept cookies
    eu_url = "https://www.euroleaguebasketball.net/euroleague/"

    driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()))
    count = 1

    driver.get(eu_url)
    delay = 10
    element = WebDriverWait(driver, delay).until(lambda x: x.find_element(By.ID, "onetrust-accept-btn-handler"))
    element.click()

    driver.get("https://www.euroleaguebasketball.net/euroleague/game-center/")
    date_element = WebDriverWait(driver, delay).until(lambda x: x.find_element(By.CLASS_NAME, "game-center-group_groupTitle__5RCk4"))
    old_date = date_element.text

    for round in range(1, 35):
        entry_url = f"https://www.euroleaguebasketball.net/euroleague/game-center/?round={round}&season=E2022"
        driver.get(entry_url)
        while not assert_round(driver, "2022-23", f"Round {round}"):
            time.sleep(1)
        
        while not old_date != driver.find_element(By.CLASS_NAME, "game-center-group_groupTitle__5RCk4").text:
            time.sleep(1)
        # s_element = WebDriverWait(driver, delay).until(lambda x: assert_round(x, "2022-23", f"Round {round}"))

        body = driver.find_element(By.TAG_NAME, "body")

        body_content = body.get_attribute("innerHTML")

        soup = bs(body_content, "html.parser")

        for link in soup.find_all("a"):
            links.append(link.get("href"))
            
    driver.close()
    link_format = r"/euroleague/game-center/2022-23/[\w-]+/E2022/[\d]{1,3}/"
    r = re.compile(link_format)
    newlist = list(filter(r.match, links))

    return list(dict.fromkeys(newlist))

In [261]:
res = load_with_selenium()

In [262]:
print(len(res))
print(res)

306
['/euroleague/game-center/2022-23/ldlc-asvel-villeurbanne-ea7-emporio-armani-milan/E2022/2/', '/euroleague/game-center/2022-23/panathinaikos-athens-real-madrid/E2022/1/', '/euroleague/game-center/2022-23/maccabi-playtika-tel-aviv-zalgiris-kaunas/E2022/3/', '/euroleague/game-center/2022-23/valencia-basket-cazoo-baskonia-vitoria-gasteiz/E2022/5/', '/euroleague/game-center/2022-23/fc-bayern-munich-fenerbahce-beko-istanbul/E2022/4/', '/euroleague/game-center/2022-23/anadolu-efes-istanbul-crvena-zvezda-meridianbet-belgrade/E2022/6/', '/euroleague/game-center/2022-23/alba-berlin-partizan-mozzart-bet-belgrade/E2022/7/', '/euroleague/game-center/2022-23/virtus-segafredo-bologna-as-monaco/E2022/9/', '/euroleague/game-center/2022-23/fc-barcelona-olympiacos-piraeus/E2022/8/', '/euroleague/game-center/2022-23/crvena-zvezda-meridianbet-belgrade-panathinaikos-athens/E2022/11/', '/euroleague/game-center/2022-23/as-monaco-anadolu-efes-istanbul/E2022/10/', '/euroleague/game-center/2022-23/virtus-se

In [277]:
def get_game_data(link):
    base_url = f"https://www.euroleaguebasketball.net{link}"
    site = requests.get(base_url)
    print(base_url)
    soup = bs(str(site.content), 'html.parser')
    columns_class = "game-box-scores-table-grouped-column_tableGroupedColumnStatGroupContainer__Oci84"
    row_class = "game-box-scores-table-grouped-column_tableGroupedRowStatGroupContainer__wluIz"
    stat_name_class = "game-box-scores-table-grouped-column_groupedTableHeading__XTxR_"
    stat_cell_class = "game-box-scores-table-grouped-column_tableStatCell__4zJlJ"

    player_link = "game-box-scores-table-grouped-tab_playerLink__MqVrl"
    re_player = r"/euroleague/players/([\w-]+)/[\d\w]+/"
    players = []
    for link in soup.find_all("a", attrs={"class": player_link}):
        players.append((link.get("href"), re.match(re_player, link.get("href")).groups()[0]))

    output_df = pl.DataFrame(data=players, schema={"links": str, "names": str})

    for element in soup.find_all("div", attrs={"class": columns_class}):
        col_stats = []
        col_heads = [stathead.text for stathead in element.find_all("div", attrs={"class": stat_name_class})]
        for row in element.find_all("div", attrs={"class": row_class}):
            stats = row.find_all("div", attrs={"class": stat_cell_class})
            if stats:
                col_stats.append([stat.text for stat in stats])
        schema = {}
        for i in col_heads:
            schema["i"]: str
        stats_df = pl.DataFrame(data=col_stats, schema=schema)
        print(stats_df)

    

    print(players)

    # link_format = r"/euroleague/game-center/2022-23/[\w-]+/E2022/[\d]{1,3}/"
    # r = re.compile(link_format)
    # newlist = list(filter(r.match, links))

    return site.content

get_game_data(res[0])

https://www.euroleaguebasketball.net/euroleague/game-center/2022-23/ldlc-asvel-villeurbanne-ea7-emporio-armani-milan/E2022/2/
shape: (2, 14)
┌──────────┬──────────┬──────────┬──────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ column_0 ┆ column_1 ┆ column_2 ┆ column_3 ┆ … ┆ column_10 ┆ column_11 ┆ column_12 ┆ column_13 │
│ ---      ┆ ---      ┆ ---      ┆ ---      ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│ str      ┆ str      ┆ str      ┆ str      ┆   ┆ str       ┆ str       ┆ str       ┆ str       │
╞══════════╪══════════╪══════════╪══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 25:34    ┆ 05:41    ┆ 32:26    ┆ 00:00    ┆ … ┆ 19:58     ┆ 24:16     ┆           ┆ 200:00    │
│ 11       ┆ 0        ┆ 6        ┆ 0        ┆ … ┆ 2         ┆ 2         ┆           ┆ 62        │
└──────────┴──────────┴──────────┴──────────┴───┴───────────┴───────────┴───────────┴───────────┘
shape: (2, 14)
┌──────────┬──────────┬──────────┬──────────┬───┬───────────

b'<!DOCTYPE html><html class="html" lang="en-GB"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width"/><title>LDLC ASVEL Villeurbanne vs EA7 Emporio Armani Milan 6 October 2022 18:00 | EuroLeague</title><meta name="google-signin-client_id" content="363904458779-buvqp49otvl4tuvuu64a4m1351d6eqin.apps.googleusercontent.com"/><link rel="canonical" href="https://www.euroleaguebasketball.net/euroleague/euroleague/game-center/2022-23/ldlc-asvel-villeurbanne-ea7-emporio-armani-milan/E2022/2/"/><meta name="description" content="Welcome to the Official Site of Euroleague Basketball. Live Matches, Stats, Standings, Teams, Players, Interviews, Fantasy Challenge, DEVOTION and much more...!!"/><meta property="og:locale" content="en_GB"/><meta property="og:title" content="LDLC ASVEL Villeurbanne vs EA7 Emporio Armani Milan 6 October 2022 18:00 | EuroLeague"/><meta property="og:site_name" content="Euroleague Basketball"/><meta property="og:image" content="https://www.eurolea